### RAG - Retrival Aughmented Generation


In [2]:
from dotenv import load_dotenv
from langchain.chains.question_answering import load_qa_chain
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    GoogleGenerativeAIEmbeddings,
)
from langchain_text_splitters import RecursiveCharacterTextSplitter

# from langchain_openai import ChatOpenAI, OpenAIEmbeddings


In [3]:
load_dotenv()

True

In [ ]:
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite", max_output_tokens=200
)

In [4]:
pdf_link = "docling.pdf"
loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True,
)
chunks = text_splitter.split_documents(pages)

In [ ]:
db = Chroma.from_documents(
    chunks, embedding=embeddings_model, persist_directory="text_index"
)

In [ ]:
# Carregar DB
vectordb = Chroma(
    persist_directory="text_index", embedding_function=embeddings_model
)
# Load Retriver
retriver = vectordb.as_retriever(search_kwargs={"k": 3})
# Construcao da cadeia de prompt para a chamada do LLM
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
def ask(question):
    context = retriever.get_relevant_documents(question)
    answer = (
        chain(
            {"imput_documents": context, "question": question},
            return_only_outputs=True,
        )
    )["output_text"]
    return answer, context

In [ ]:
user_question = input("User: ")
answer, context = ask(user_question)
print("Answer: ", answer)

In [ ]:
print(context[0])